# Training

In [ ]:
from pathlib import Path
from dataclasses import dataclass
import sys

# Add the root project directory to the Python path
ROOT = Path.cwd().parent  # This will get the project root since the notebook is in 'notebooks/'
sys.path.append(str(ROOT))
from configs.path_config import EXTRACTED_DATA_DIR, WEIGHTS_DIR
from src.processing import dataset
from models import lstm_model
from src import utils

#### Create the train data loader

In [ ]:
folder_path = EXTRACTED_DATA_DIR / 'group2_south'
data = dataset.StrainDataset(folder_path, sequence_length=128, start_idx=11112, test_size=0.3)
train_loader = data.train_dataloader

#### Train the model

In [ ]:
# Model parameters
input_dim = data.feature_count
hidden_dim=32
num_layers=2
num_epochs=5
learning_rate=0.001


# Location and name for saving the model
model_folder = WEIGHTS_DIR
model_name = f'lstm_model_{input_dim}_{hidden_dim}_{num_layers}_{num_epochs}_{learning_rate}.pth'

# Create the model and train it
model = lstm_model.LSTMModel(input_dim, hidden_dim, num_layers)
losses, prediction = lstm_model.training_loop(model, train_loader, num_epochs, learning_rate, model_folder, model_name)

#### Plot the epoch losses

In [ ]:
utils.plot_epochs_loss(num_epochs, losses)